### Consigna del desafío 1

**1**. Vectorizar documentos. Tomar 5 documentos al azar y medir similaridad con el resto de los documentos.
Estudiar los 5 documentos más similares de cada uno analizar si tiene sentido
la similaridad según el contenido del texto y la etiqueta de clasificación.

**2**. Entrenar modelos de clasificación Naïve Bayes para maximizar el desempeño de clasificación
(f1-score macro) en el conjunto de datos de test. Considerar cambiar parámteros
de instanciación del vectorizador y los modelos y probar modelos de Naïve Bayes Multinomial
y ComplementNB.

**3**. Transponer la matriz documento-término. De esa manera se obtiene una matriz
término-documento que puede ser interpretada como una colección de vectorización de palabras.
Estudiar ahora similaridad entre palabras tomando 5 palabras y estudiando sus 5 más similares. **La elección de palabras no debe ser al azar para evitar la aparición de términos poco interpretables, elegirlas "manualmente"**.


In [56]:
import random
from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.naive_bayes import MultinomialNB, ComplementNB, BernoulliNB
from sklearn.metrics import f1_score

import pandas as pd
import numpy as np

In [3]:
# cargamos los datos (ya separados de forma predeterminada en train y test)
newsgroups_train = fetch_20newsgroups(subset='train', remove=('headers', 'footers', 'quotes'))
newsgroups_test = fetch_20newsgroups(subset='test', remove=('headers', 'footers', 'quotes'))

# en el atributo `data` accedemos al texto
newsgroups_train.data[0]

'I was wondering if anyone out there could enlighten me on this car I saw\nthe other day. It was a 2-door sports car, looked to be from the late 60s/\nearly 70s. It was called a Bricklin. The doors were really small. In addition,\nthe front bumper was separate from the rest of the body. This is \nall I know. If anyone can tellme a model name, engine specs, years\nof production, where this car is made, history, or whatever info you\nhave on this funky looking car, please e-mail.'

In [24]:
# instanciamos un vectorizador
tfidfvect = TfidfVectorizer()

# con la interfaz habitual de sklearn podemos fitear el vectorizador
# (obtener el vocabulario y calcular el vector IDF)
# y transformar directamente los datos
X_train = tfidfvect.fit_transform(newsgroups_train.data)
# `X_train` la podemos denominar como la matriz documento-término

y_train = newsgroups_train.target

# recordar que las vectorizaciones por conteos son esparsas
# por ello sklearn convenientemente devuelve los vectores de documentos
# como matrices esparsas
print(type(X_train))
print(f'shape: {X_train.shape}')
print(f'cantidad de documentos: {X_train.shape[0]}')
print(f'tamaño del vocabulario (dimensionalidad de los vectores): {X_train.shape[1]}')

<class 'scipy.sparse._csr.csr_matrix'>
shape: (11314, 101631)
cantidad de documentos: 11314
tamaño del vocabulario (dimensionalidad de los vectores): 101631


In [61]:
#vamos a ver el balanceo entre clases
class_labels = newsgroups_train.target_names

# Contar la cantidad de documentos en cada clase
class_counts = np.bincount(y_train)

# Imprimir la cantidad de documentos en cada clase
for label, count in zip(class_labels, class_counts):
    print(f'{label}: {count}')

alt.atheism: 480
comp.graphics: 584
comp.os.ms-windows.misc: 591
comp.sys.ibm.pc.hardware: 590
comp.sys.mac.hardware: 578
comp.windows.x: 593
misc.forsale: 585
rec.autos: 594
rec.motorcycles: 598
rec.sport.baseball: 597
rec.sport.hockey: 600
sci.crypt: 595
sci.electronics: 591
sci.med: 594
sci.space: 593
soc.religion.christian: 599
talk.politics.guns: 546
talk.politics.mideast: 564
talk.politics.misc: 465
talk.religion.misc: 377


In [40]:
# Seleccionar 5 documentos aleatorios
#random_docs_indices = random.sample(range(X_train.shape[0]), 5)
random_docs_indices = [600 , 8820 , 8770 , 7846, 10913]
random_docs = X_train[random_docs_indices]

# Mostrar los documentos seleccionados
for i, doc_idx in enumerate(random_docs_indices):
    print(f"Document {doc_idx} content:\n{newsgroups_train.data[doc_idx]}\n")

Document 600 content:
I have been looking for a book that specifically addresses
the mystery of God in the paradox.  I have read some that touch
on the subject in a chapter but would like a more detailed read.

Is anyone aware of any books that deal with this subject.

Please e-mail me.  Thanks.

Document 8820 content:
I have an external hard drive I wish to use as startup disk.
problem is, when I switch on the mac, it boots on the internal HD,
but when I restart  (warm boot) the mac, it boots from the external.
how do I make  it boot directly from the external ?

please email replies if possible.

thanks in advance.

Document 8770 content:

If you are really interested in these orbits and how they are obtained
you should try and find the following paper:

     Hiroshi Yamakawa, Jun'ichiro Kawaguchi, Nobuaki Ishii, 
     and Hiroki Matsuo, "A Numerical Study of Gravitational Capture
     Orbit in the Earth-Moon System," AAS-92-186, AAS/AIAA Spaceflight
     Mechanics Meeting, Colorado 

In [41]:
# Calcular similitud
similarity_matrix = cosine_similarity(random_docs, X_train)

# Encontrar los 5 documentos más similares para cada documento seleccionado
most_similar_docs = np.argsort(-similarity_matrix, axis=1)[:, 1:6]


# Imprimir los nombres de las categorías de los documentos más similares
for i, doc_idx in enumerate(random_docs_indices):
    print(f"Document {doc_idx} category: {newsgroups_train.target_names[y_train[doc_idx]]}")
    print("Most similar documents categories:")
    for similar_doc_idx in most_similar_docs[i]:
        print(f"\t- {similar_doc_idx}: {newsgroups_train.target_names[y_train[similar_doc_idx]]}")


Document 600 category: soc.religion.christian
Most similar documents categories:
	- 10836: alt.atheism
	- 5826: soc.religion.christian
	- 7811: comp.sys.ibm.pc.hardware
	- 8475: alt.atheism
	- 2785: alt.atheism
Document 8820 category: comp.sys.mac.hardware
Most similar documents categories:
	- 6221: comp.sys.mac.hardware
	- 9624: comp.sys.mac.hardware
	- 3390: comp.sys.ibm.pc.hardware
	- 5955: comp.sys.mac.hardware
	- 4520: rec.motorcycles
Document 8770 category: sci.space
Most similar documents categories:
	- 1176: sci.space
	- 8525: sci.space
	- 1511: talk.politics.misc
	- 762: sci.space
	- 10344: sci.space
Document 7846 category: comp.windows.x
Most similar documents categories:
	- 8757: comp.windows.x
	- 6099: comp.windows.x
	- 4614: comp.windows.x
	- 2851: comp.windows.x
	- 8887: comp.windows.x
Document 10913 category: sci.crypt
Most similar documents categories:
	- 10344: sci.space
	- 6880: talk.politics.misc
	- 5423: talk.politics.mideast
	- 913: alt.atheism
	- 1191: talk.politi

In [54]:
print(newsgroups_train.data[10836])
print("------------####-----------")
print(newsgroups_train.data[7811])

Archive-name: atheism/faq
Alt-atheism-archive-name: faq
Last-modified: 5 April 1993
Version: 1.1

                    Alt.Atheism Frequently-Asked Questions

This file contains responses to articles which occur repeatedly in
alt.atheism.  Points covered here are ones which are not covered in the
"Introduction to Atheism"; you are advised to read that article as well
before posting.

These answers are not intended to be exhaustive or definitive. The purpose of
the periodic FAQ postings is not to stifle debate, but to raise its level. If
you have something to say concerning one of these questions and which isn't
covered by the answer given, please feel free to make your point.

Overview of contents:

   "What is the purpose of this newsgroup?"
   "Hitler was an atheist!"
   "The Bible proves it"
   "Pascal's Wager"
   "What is Occam's Razor?"
   "Why it's good to believe in Jesus"
   "Why I know that God exists"
   "Einstein and "God does not play dice""
   "Everyone worships something"


El texto trata de religión y especificamente pregunta si alguien tiene un libro. El texto mas similar es bastente largo y son preguntas y respuestas de religion. 
Por curiosidad decidi ver el documento 7811, clase: comp.sys.ibm.pc.hardware, y busca libros para solucionar un problema de hardware, lo cual coincide con la acción del documento base, aunque el tema es distinto.

In [50]:
for similar_doc_idx in most_similar_docs[i][:2]:  # Imprimir solo los 2 más similares
        print(f"\t- Document {similar_doc_idx}: {newsgroups_train.target_names[y_train[similar_doc_idx]]}")
        print(f"\t  Content: {newsgroups_train.data[similar_doc_idx]}...") 


	- Document 10344: sci.space
	  Content: ETHER IMPLODES 2 EARTH CORE, IS GRAVITY!!!

  This paper BOTH describes how heavenly bodys can be stationary, 
ether sucking structures, AND why we observe "orbital" motion!!

  Ether, the theoretical propogation media of electro-magnetic 
waves, was concluded not to exist, based on the results of the 
Michelson-Moreley experiment conducted a century ago. 

  I propose that those conclusions are flawed, based on the fact 
that the experiment was designed to look for a flow parallel to the 
earth's surface, not perpindicular. (Due to the prevailing assumption 
that the earth traveled through the ether as a ball through the wind)

  The reversal of the that conclusion, a pivotal keystone in the 
development of modern scientific thought, could have ramifications 
of BIBLICAL proportions through out the WORLD!!

  REMEMBER: Einstien said Imagination is greater than knowledge!!

  
1
  I dream like this: ether based reality
 
  The ether is like a fl

**Punto 2**

In [55]:
clf = MultinomialNB()
clf.fit(X_train, y_train)

# con nuestro vectorizador ya fiteado en train, vectorizamos los textos
# del conjunto de test
X_test = tfidfvect.transform(newsgroups_test.data)
y_test = newsgroups_test.target
y_pred =  clf.predict(X_test)

f1_score(y_test, y_pred, average='macro')

0.5854345727938506

In [57]:
# Definir los modelos de Naïve Bayes
models = {
    "MultinomialNB": MultinomialNB(),
    "ComplementNB": ComplementNB(),
    "BernoulliNB": BernoulliNB()
}

# Entrenar y evaluar los modelos
for model_name, model in models.items():
    model.fit(X_train, y_train)
    
    # Vectorizar los textos del conjunto de test
    X_test = tfidfvect.transform(newsgroups_test.data)
    y_test = newsgroups_test.target
    
    # Predecir con el modelo entrenado
    y_pred = model.predict(X_test)
    
    # Calcular el f1-score
    f1 = f1_score(y_test, y_pred, average='macro')
    print(f"{model_name} F1 Score: {f1}")

MultinomialNB F1 Score: 0.5854345727938506
ComplementNB F1 Score: 0.692953349950875
BernoulliNB F1 Score: 0.42866948824103135


ComplementNB dio el mejor score y por lo tanto parece una buena opción para este conjunto de datos. Ademas hay cierto desbalanceo entre clases y este modelo esta diseñado para estas situaciones.

**Punto 3**

In [66]:
# transponemos la matriz documento-término
X_transposed = X_train.transpose()

selected_words = ['friend', 'house', 'magic', 'sports', 'power']
selected_word_indices = [tfidfvect.vocabulary_.get(word, -1) for word in selected_words]
selected_word_indices = [idx for idx in selected_word_indices if idx != -1]

# Calcular similitud entre palabras
word_similarity_matrix = cosine_similarity(X_transposed[selected_word_indices], X_transposed)

# Encontrar las 5 palabras más similares para cada palabra seleccionada
most_similar_words = np.argsort(-word_similarity_matrix, axis=1)[:, 1:6]

# Analizar similitud
for i, word_idx in enumerate(selected_word_indices):
    print(f"Word '{selected_words[i]}' most similar words: {most_similar_words[i]}")
    for similar_word_idx in most_similar_words[i]:
        similar_word = tfidfvect.get_feature_names_out()[similar_word_idx]
        print(f" - {similar_word}")

Word 'friend' most similar words: [61856 82513 76015 50826 37802]
 - mine
 - shyness
 - rang
 - invite
 - err
Word 'house' most similar words: [96497 81580 30287 62197 32941]
 - white
 - senate
 - cpr
 - miyazawa
 - deposit
Word 'magic' most similar words: [60792 36785 79489 69872 56077]
 - meguire
 - elixir
 - rubbing
 - param
 - lectric
Word 'sports' most similar words: [96855 52498 72249 78987 16930]
 - wip
 - jockeys
 - pollute
 - rockin
 - admittdly
Word 'power' most similar words: [86523 88532 21710 72103 12308]
 - supply
 - the
 - bartmich
 - pocatello
 - 8039


Vemos que hay poca/nada relación semantica entre las palabras buscadas y sus similares ya que el modelo TF-IDF se basa en la frecuencia de términos y no en el contexto en el que aparecen las palabras.